# Batch Normalization의 활용성 정리
### 이전내용 정리 (Tut 10~13)

### 인공신경망의 문제점

보통 인공신경망이라 하면 대게 입력층 - 은닉층 - 출력층으로 나눠져 있는 fully connected layer를 많이들 떠올립니다. 이렇게 구조화된 신경망은 연산을 할때 내적행렬곱을 통해 weighted sum을 계산후 비선형 함수를 적용시켜 아웃풋값들을 출력하게 됩니다. 이 계산과정을 각 레이어마닫 거친다 하여 __feedforward__ 라고 합니다, 이 과정시 hyperparameter 들이 출력층에서 지대한 영향을 미치게 됩니다. 순전파 인공신경망에서 hyperparmeter란 은닉층의 갯수와 은닉층당 뉴런의 갯수를 뜻합니다. 이런 hyperparmeter들을 계속해서 튜닝함과 동시에 오차 역전파를 통해 인공신경망은 알맞는 값을 내어놓습니다. 인공신경망에서 hyperparmeter들은 여러 복잡한 계산을 가능케 하는데 이 hyperparmeter가 너무 많으면 이전에 배운 overfitting 문제를 야기 시킵니다.  

과적함이란 training dataset에 신경망이 너무 너무 과하게 맞춰서 학습이된 상태를 말합니다. 결국 training accuarcy는 높을지 몰라도 validation 과정시 validation data가 training data패턴에 없던 data이면 신경망은 벗어난 값을 예측하게 됩니다.  

결국 generalization의 큰 문제가 생기게 됩니다. 이러한 과적합 문제 외에도 인공신경망은 hyperparmeter에 대한 여러가지 문제를 가지고 있습니다. __또다른 문제점으로 심층 신경망에서 순전파 계산시 은닉 뉴런들을 연결하는 가중치들중 몇몇값들이 펴균 가중치값보다 클 경우 계속해서 weighted sum이 되어 각 레이어의 분포가 달라지게 됩니다__ 이것을 우리는 __Internal Covariate Shift__ 현상이라고 하였습니다.   
쉽게 말해 input distribution이 각 레이어 마다 다르다는 뜻 입니다.  
레이어가 뒤로 갈수록 분포가 변화되어 결국 출력층에 안좋은 영향을 끼칠수 있게 됩니다. Vanishing Gradient 혹은 Gradient Explosion등이 이러한 문제가 될수 있습니다. 이러한 현상을 방지하고자 논문에서는 각 네트워크 망의 distribution을 같게 하는 방법을 고안하였고 이것을 저자는 Batch Normalization (배치 정규화)라고 소개합니다.  
배치정규화의 이점은 학습속도의 향상과 자체적인 regularization이라고 합니다.  
https://arxiv.org/pdf/1502.03167.pdf   


**내적행렬곱??**

### 배치 정규화 알고리즘

인공신경망의 문제점을 해결하기 위해 많은 연구들이 있었습니다.  
Vanishing Gradient 현상을 없애기 위해 __Rectified linear unit (ReLU)__ 활성화 함수를 고안하고, 평균보다 큰 가중치들을 줄이기 위해 L-2 혹은 L-1 __regularizer weight decay기법__ , 그리고 각 레이어마다 랜덤으로 뉴런을 꺼버려 노이즈를 발생시켜 가중치의 편향을 방지해주는 __Dropout__ 기법등, 다양한 방법들로 인공신경망의 문제점들을 개선하려 여러 노력을 해왔습니다.  
이런 방법들은 대게 데이터에다 직접 손을 대지 않는 간접적인 해결책이라고 볼수 있습니다. 
하지만 Batch Normalization기법은 네트워크망 내부 데이ㅓ를 아예 안정적으로 학습을 할 수 있도록 만들어 줍니다. 논문에서 데이터가 whited, linearly transformed to zero mean, unit variance, decorrelated 시에 coverage를 빨리 한다고 합니다. 결국 평균0 표준편차 1인 분포로 만들면 된다는것입니다. 

![](https://shuuki4.files.wordpress.com/2016/01/bn1.png)


과정을 살펴보면 미니배치의 평균과 각 데이터와의 거리를 제곱한것의 합의 평균 즉 즉, 분산(variance)를 구합니다. 그리고 각 데이터와 미니배치의 평균의 거리를 분산에 numerical stability를 위해 epsilon이라는 미세한 값을 더한것의 제곱근으로 나누어줍니다. (NaN을 예방), 여기서 끝이 아니라 gamma값을 곱하고 beta값을 더해줌으로써 scale과 shift를 합니다. 이것은 gamma와 beta를 trainable하게 하는 파라미터로써 손실함수를 각 파라미터에 대해서 편미분을 하고 경사하강법으로 gamma값과 beta값을 extra layer로 볼수 있습니다. 아래 사진은 어파인 layer 값들을 활성화 함수로 넘겨주기전에 정규화를 한후 비선형 활성화 함수를 적용해본 것입니다.

![](https://i0.wp.com/mohammadpz.github.io/posts/2015_02_01_IFT6266_Cats_vs_Dogs/img/bn.png)

### 배치 정규화의 이점

Batch Normalization의 가장 궁극적인 목표는 internal covariagte shift를 줄이는 것입니다. 신경망의 각 레이어의 분포를 같게 함으로써 좀더 안정적인 학습이 가능해집니다. 하지만 더 나아가 배치정규화를 사용하면 추가로 2가지 이점이 더 있다고 합니다.  

1. 빠른 학습속도
보통 경사하강법을 통해 학습을 하려고 보면 학습률(learning rate)를 고민할때가 많습니다. 너무 작게 하면 학습시간이 너무 오래 걸립니다. 그렇다고 너무 높게 잡으면 gradient vanishing현상이나 exploding현상이 발생하게 됩니다. 하지만 배치 정규화를 넣으면 parameter들의 scale에 학습률이 크게 작용을 하지 않기 때문에 높은 학습률 사용이 가능하게 됩니다. 그래서 학습이 비교적 빠르다고 할수 있습니다.  

2. 자체적인 regularization
논문은 배치 정규화를 사용함으로써 자체적인 regularization효과가 있다고 말합니다. 따라서 weight decay나 dropout등을 굳이 쓰지 않아도 말합니다.  